<a href="https://colab.research.google.com/github/billgatos/IA25_P01_G03/blob/main/IA_P01_G03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-constraint matplotlib numpy psutil nvidia-ml-py3

### Importação de Bibliotecas

Importação das bibliotecas necessárias.

In [ ]:
import os
import random
from typing import Dict, List, Tuple, Any
from constraint import *
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from google.colab import files
import psutil
import threading
import time
from pynvml import *

### Função de Carregamento de Dados

Esta função serve para lermos o ficheiro com as informações das turmas, professores, disciplinas, aulas e restrições.
Transformar esse conteúdo num dicionário organizado em várias categorias.

In [ ]:
# --- FUNÇÃO DE CARREGAMENTO DE DADOS (AJUSTADA) ---
# Esta função serve para ler um ficheiro de texto (por exemplo, "ClassTT_01_tiny.txt")
# e transformar o conteúdo dele num dicionário organizado com várias categorias
# de informação sobre horários, professores, cursos, salas, etc.

def load_timetable_data(filepath: str) -> Dict[str, Any]:
    """
    Lê e processa os dados de definição do horário a partir de um arquivo de texto
    no formato do ClassTT_01_tiny.txt.
    """

    # Cria um dicionário vazio com várias chaves que vão guardar diferentes tipos de dados
    data = {
        'CLASS_COURSES': {},              # Mapeia uma turma para as disciplinas que ela tem
        'TEACHER_COURSES': {},            # Mapeia um professor para as disciplinas que ele leciona
        'COURSE_ROOM_RESTRICTIONS': {},   # Guarda restrições de salas por disciplina
        'TEACHER_UNAVAILABLE_SLOTS': {},  # Guarda horários em que cada professor não está disponível
        'ONLINE_LESSONS': {},             # Indica quais aulas são online e em que semana
        'COURSE_TEACHERS': {}             # Mapeia disciplinas para professores (é o inverso de TEACHER_COURSES)
    }

    # Variável para saber em que parte (seção) do ficheiro estamos a ler
    current_section = None

    try:
        # Abre o ficheiro no caminho indicado (modo leitura)
        with open(filepath, 'r') as f:
            # Lê o ficheiro linha a linha
            for line in f:
                # Remove espaços em branco no início e fim da linha
                line = line.strip()

                # Ignora linhas vazias, linhas de cabeçalho e comentários que não contêm dados
                if not line or line.startswith('#head') or line.startswith('—') or line.startswith('(class'):
                    continue

                # Quando encontra uma linha que começa com '#', isso indica o início de uma nova seção
                if line.startswith('#'):
                    # Guarda o nome da seção (por exemplo, '#cc' ou '#dsd')
                    # Assim sabemos que tipo de dados vêm a seguir
                    current_section = line.split(' ')[0]
                    continue

                # Divide a linha em partes (separadas por espaços)
                parts = line.split()
                if not parts:
                    continue

                # A primeira parte geralmente é a chave (ex: nome de turma, professor ou disciplina)
                key = parts[0]
                # O resto da linha são os valores associados (ex: disciplinas, horários, etc.)
                values = parts[1:]

                # A partir daqui, a função decide o que fazer com base na seção atual:
                # Cada bloco "if" abaixo trata de um tipo de informação diferente.

                if current_section == '#cc':
                    # Seção "#cc" — (class, courses*)
                    # Exemplo: 10A MAT PORT ENG -> a turma 10A tem as disciplinas MAT, PORT, ENG
                    data['CLASS_COURSES'][key] = values

                elif current_section == '#dsd':
                    # Seção "#dsd" — (teacher, courses*)
                    # Exemplo: Joao MAT ENG -> o professor Joao dá MAT e ENG
                    data['TEACHER_COURSES'][key] = values

                elif current_section == '#tr':
                    # Seção "#tr" — (teacher, slots_unavailable*)
                    # Lista de horários em que o professor não pode dar aula
                    try:
                        # Converte cada valor da lista em inteiro (porque os horários são números)
                        data['TEACHER_UNAVAILABLE_SLOTS'][key] = [int(v) for v in values]
                    except ValueError:
                        # Se algum valor não for número, mostra um aviso mas não quebra o código
                        print(f"Aviso: Ignorando slots indisponíveis não numéricos para o professor {key}.")

                elif current_section == '#rr':
                    # Seção "#rr" — (course, room)
                    # Exemplo: MAT Sala1 -> disciplina MAT tem de ser dada na Sala1
                    if values:
                        data['COURSE_ROOM_RESTRICTIONS'][key] = values[0]  # Só interessa o primeiro valor (a sala)

                elif current_section == '#oc':
                    # Seção "#oc" — (course, lesson_week_index)
                    # Exemplo: ENG 2 -> a segunda aula de ENG é online
                    if values:
                        try:
                            # Converte o valor em número (1 ou 2)
                            data['ONLINE_LESSONS'][key] = int(values[0])
                        except ValueError:
                            # Caso tenha algo inválido, avisa o utilizador
                            print(f"Aviso: Índice de lição online não numérico para a disciplina {key}.")

    # Se o ficheiro não for encontrado, mostra uma mensagem de erro e interrompe o programa
    except FileNotFoundError:
        print(f"Erro: Arquivo {filepath} não encontrado. O script não pode continuar sem dados.")
        raise FileNotFoundError(f"Arquivo de dados '{filepath}' não encontrado.")

    # Aqui fazemos o "inverso" de TEACHER_COURSES -> COURSE_TEACHERS
    # Isso é útil para poder consultar rapidamente qual professor ensina determinada disciplina
    for teacher, courses in data['TEACHER_COURSES'].items():
        for course in courses:
            data['COURSE_TEACHERS'][course] = teacher

    # Por fim, devolve o dicionário completo com todos os dados processados
    return data


### Funções de Visualização

Estas funções transformam os blocos numéricos (1–20) em dias e horários legíveis, e permitem desenhar
o horário completo de uma turma com `matplotlib`.  
A escolha desta abordagem facilita a análise dos resultados, tornando o horário mais intuitivo
e permitindo verificar rapidamente a distribuição das aulas e possíveis conflitos.


In [ ]:
# --- FUNÇÕES AUXILIARES PARA VISUALIZAÇÃO ---
# Estas funções ajudam a traduzir "blocos" do horário (1..20) para dia/slot
# e a desenhar um horário bonitinho com matplotlib.

def get_day(block):
    """Mapeia o bloco (1-20) para o dia da semana."""
    # Dividimos por 4 porque assumimos 4 blocos por dia.
    # (block - 1) para converter de 1-based para 0-based antes da divisão inteira.
    # DAYS deve ser algo como ['Seg', 'Ter', 'Qua', 'Qui', 'Sex'] e existir no escopo.
    return DAYS[(block - 1) // 4]

def block_to_day_time(block_num):
    """Mapeia o número do bloco (1-20) para o dia e o slot de tempo do dia."""
    BLOCKS_PER_DAY = 4  # Assumimos que cada dia tem 4 blocos (S1..S4)

    # Índice do dia na lista DAYS (0..len(DAYS)-1)
    day_index = (block_num - 1) // BLOCKS_PER_DAY
    # Índice do slot dentro do dia (0..3). O resto da divisão dá o slot.
    time_slot_index = ((block_num - 1) % BLOCKS_PER_DAY)

    # Gera labels dos slots: ['S1', 'S2', 'S3', 'S4']
    TIME_LABELS = [f'S{i+1}' for i in range(BLOCKS_PER_DAY)]

    # Devolve: nome do dia (ex.: 'Seg'), label do slot (ex.: 'S2'),
    # e também os índices numéricos, caso sejam úteis a montagens de matrizes.
    return DAYS[day_index], TIME_LABELS[time_slot_index], day_index, time_slot_index

def display_timetable(solutions, class_id, CLASS_COURSES, COURSE_TEACHERS):
    """
    Desenha o horário para uma turma específica usando matplotlib.
    - solutions: dicionário com variáveis de decisão já resolvidas (ex.: {'10A_MAT_0': (bloco, sala), ...})
    - class_id: a turma a desenhar (ex.: '10A')
    - CLASS_COURSES: mapeia turma -> lista de disciplinas
    - COURSE_TEACHERS: mapeia disciplina -> professor
    """
    BLOCKS_PER_DAY = 4                         # Mesmo pressuposto: 4 blocos por dia
    TIME_LABELS = [f'S{i+1}' for i in range(BLOCKS_PER_DAY)]  # Labels verticais (S1..S4)

    # Matriz base para o heatmap: linhas = slots, colunas = dias
    # Começa tudo a zero (0 quer dizer “sem disciplina atribuída”)
    timetable_data = np.zeros((BLOCKS_PER_DAY, len(DAYS)))
    # Matriz de strings com o que vai escrito dentro de cada célula (curso/prof/sala)
    text_labels = np.full((BLOCKS_PER_DAY, len(DAYS)), '', dtype=object)

    # Lista de TODAS as disciplinas existentes nas turmas (para colorir consistentemente)
    all_courses = sorted(list(set(c for courses in CLASS_COURSES.values() for c in courses)))
    # Mapeia disciplina -> índice numérico (1..N). 0 ficou reservado para “vazio”.
    course_to_index = {course: i + 1 for i, course in enumerate(all_courses)}

    # 2. Processa as variáveis da turma específica
    for var, value in solutions.items():
        # Só olhamos para variáveis que começam pelo id da turma, ex.: '10A_...'
        if var.startswith(f'{class_id}_'):

            # value esperado como tupla: (block_num, room)
            block_num, room = value
            # Nome da variável tipicamente no formato '{class}_{course}_{...}'
            parts = var.split('_')
            course = parts[1]  # a segunda parte costuma ser a disciplina

            # Quem dá esta disciplina? Se não estiver mapeado, colocamos 'N/A'
            teacher = COURSE_TEACHERS.get(course, 'N/A')

            # Convertemos o bloco absoluto (1..20) em (dia, slot) e respetivos índices
            day, time_label, day_idx, time_slot_idx = block_to_day_time(block_num)

            # Texto que vai dentro da célula (3 linhas: disciplina, professor, sala)
            text_labels[time_slot_idx, day_idx] = f'{course}\n({teacher})\n[{room}]'

            # Preenchemos a célula numérica com o índice da disciplina (para dar cor no heatmap)
            timetable_data[time_slot_idx, day_idx] = course_to_index.get(course, 0)

    # 3. Desenho com Matplotlib

    # Cria a figura e o eixo. Tamanho 12x6 costuma ficar equilibrado num ecrã normal.
    fig, ax = plt.subplots(figsize=(12, 6))

    # Escolhemos uma paleta discreta com bastantes categorias (tab20) + 1 para o "vazio"
    cmap = cm.get_cmap('tab20', len(all_courses) + 1)

    # Desenha o “heatmap” principal: cada célula representa (slot, dia)
    im = ax.imshow(timetable_data, cmap=cmap, aspect='auto')

    # Configura os eixos para bater certo com a grelha: X = dias, Y = slots
    ax.set_xticks(np.arange(len(DAYS)))
    ax.set_yticks(np.arange(BLOCKS_PER_DAY))
    ax.set_xticklabels(DAYS)          # Nomes dos dias por coluna
    ax.set_yticklabels(TIME_LABELS)   # S1..S4 por linha

    # Títulos e legendas dos eixos para ficar claro ao leitor
    plt.xlabel("Dias da Semana")
    plt.ylabel("Slots de Tempo (S1 = Bloco 1, S4 = Bloco 4)")
    plt.title(f"Horário Resolvido para a Turma: {class_id}", fontsize=14)

    # Escreve o texto em cada célula que tiver conteúdo (disciplina/prof/sala)
    for i in range(BLOCKS_PER_DAY):
        for j in range(len(DAYS)):
            if text_labels[i, j]:
                # Preto costuma ter bom contraste com a paleta tab20 clara
                text_color = "black"
                ax.text(j, i, text_labels[i, j],
                        ha="center", va="center", color=text_color, fontsize=9, fontweight='bold')

    # Desenha as linhas da grelha para parecer uma tabela (bordas entre células)
    ax.set_xticks(np.arange(len(DAYS)+1)-.5, minor=True)
    ax.set_yticks(np.arange(BLOCKS_PER_DAY+1)-.5, minor=True)
    ax.grid(which="minor", color="k", linestyle='-', linewidth=2)
    ax.tick_params(which="minor", bottom=False, left=False)

    # Mostra a figura. (Se estiveres em notebook, aparece inline; em script, abre janela.)
    plt.show()

# --- FIM DAS FUNÇÕES DE VISUALIZAÇÃO ---


### Monitorização de Desempenho

Esta função foi criada para acompanhar o uso de CPU e memória durante a execução do agente, permitindo avaliar o impacto do processamento na máquina.  
O monitor regista a percentagem de utilização de CPU e RAM, guardando essas leituras num dicionário indexado pelo tempo decorrido.  
Decidimos incluir esta componente porque o problema de geração de horários pode envolver bastantes combinações e consumir recursos significativos, sobretudo com datasets maiores.  
A monitorização ajuda-nos a perceber se o algoritmo está a ser eficiente e a comparar o desempenho entre diferentes versões.


In [ ]:
def monitor_performance(interval=0.5):
    """Monitor de CPU e memória universal"""
    # Cria um dicionário vazio onde vamos guardar as leituras ao longo do tempo.
    # Cada chave será o tempo decorrido (em segundos), e o valor será outro dicionário com CPU e memória.
    performance_data = {}

    # Marca o momento em que começamos a monitorizar.
    start = time.time()

    # Variável de controlo para o loop; enquanto for True, continuamos a medir.
    running = True

    # Loop principal de monitorização
    while running:
        # Calcula o tempo decorrido desde o início (arredondado a 2 casas decimais)
        t = round(time.time() - start, 2)

        # Mede a percentagem de uso da CPU no instante atual.
        # O argumento interval=None faz com que a função devolva o valor “instantâneo” (sem esperar).
        cpu = psutil.cpu_percent(interval=None)

        # Mede o uso total de memória RAM em percentagem.
        mem = psutil.virtual_memory().percent

        # Guarda os valores lidos num dicionário com o tempo como chave.
        # Exemplo: {0.5: {'cpu': 22.3, 'mem': 57.1}, 1.0: {...}, ...}
        performance_data[t] = {"cpu": cpu, "mem": mem}

        # Espera o tempo definido no argumento “interval” antes da próxima medição.
        # Por omissão, meio segundo (0.5s).
        time.sleep(interval)

        # Esta parte serve só para o exemplo — o loop pára automaticamente após 5 segundos.
        # Na prática, poderias trocar isto por uma condição externa ou sinal de interrupção.
        if t > 5:
            running = False

    # No fim, devolve o dicionário completo com todas as medições recolhidas.
    return performance_data


### Modelação e Resolução do CSP

Nesta fase do projeto modelámos o problema como um **CSP**, em que cada aula é tratada como uma variável associada a combinações possíveis de bloco horário e sala.  
Optámos por criar um domínio universal com todos os blocos e salas, aplicando depois as restrições necessárias para reduzir o espaço de procura.  
Foram consideradas restrições que impedem sobreposições de aulas para turmas, professores e salas, e que distribuem as aulas de uma mesma disciplina por dias diferentes.  
O solver da biblioteca `python-constraint` é então utilizado para procurar uma solução viável e, quando encontrada, o sistema gera automaticamente os horários correspondentes.

In [ ]:
# --- CSP MODELAGEM E SOLUÇÃO ---
# Aqui definimos a extração de info das variáveis e montamos/solucionamos o CSP.

# Helper function to get information from the variable string
def get_lesson_info(variable_name):
    # As variáveis vêm no formato 'Class_Course_Lx' (ex.: 't01_UC11_L1')
    parts = variable_name.split('_')         # Divide em ['Class', 'Course', 'L1']
    class_id = parts[0]                      # A turma (ex.: 't01')
    course = parts[1]                        # A disciplina (ex.: 'UC11')
    lesson_index = int(parts[2][1:])         # 'L1' -> 1, 'L2' -> 2 (corta o 'L' e converte)
    teacher = COURSE_TEACHERS.get(course)    # Procura o professor que dá esta disciplina
    return class_id, course, lesson_index, teacher

def solve_and_display():

    # Validação inicial de dados
    # Se não houver dados de turmas/disciplinas, não vale a pena prosseguir.
    if not CLASS_COURSES:
        print("\nO CSP não pode ser resolvido pois não há turmas/disciplinas carregadas.")
        return

    problem = Problem()  # Instancia o problema CSP (do python-constraint)

    # Variables are of the form 'Class_Course_LessonIndex' (e.g., 't01_UC11_L1')
    LESSON_VARIABLES = []  # Lista com TODAS as variáveis de aula a agendar
    for class_id, courses in CLASS_COURSES.items():
        for course in courses:
            # Cada par turma-disciplina tem 2 aulas (L1 e L2)
            LESSON_VARIABLES.append(f'{class_id}_{course}_L1')
            LESSON_VARIABLES.append(f'{class_id}_{course}_L2')

    # The Universal Domain: All possible (Time_Block, Room) combinations
    # Domínio completo: todos os blocos temporais possíveis cruzados com todas as salas possíveis
    UNIVERSAL_DOMAIN = [(block, room) for block in BLOCKS for room in ROOMS]

    # 4a. Add Variables with Restricted Domains
    # Para cada variável (aula), começamos do domínio universal e vamos cortando
    # conforme as restrições (professor indisponível, sala obrigatória, aula online, ...)

    for var in LESSON_VARIABLES:
        class_id, course, lesson_index, teacher = get_lesson_info(var)

        restricted_domain = list(UNIVERSAL_DOMAIN)  # Começa com tudo

        # 1. Apply Teacher Time Restrictions (tr)
        # Remove blocos em que o professor não pode dar aula
        if teacher in TEACHER_UNAVAILABLE_SLOTS:
            unavailable_slots = TEACHER_UNAVAILABLE_SLOTS[teacher]
            restricted_domain = [
                (block, room) for block, room in restricted_domain
                if block not in unavailable_slots
            ]

        # 2. Apply Room Restrictions (rr)
        # Se a disciplina exige uma sala específica, filtra só essa sala
        required_room = COURSE_ROOM_RESTRICTIONS.get(course)
        if required_room:
            restricted_domain = [
                (block, room) for block, room in restricted_domain
                if room == required_room
            ]

        # 3. Apply Online Class Constraint (oc)
        # Se esta aula específica (L1 ou L2) é online, força a sala 'Online'
        if lesson_index == ONLINE_LESSONS.get(course):
            restricted_domain = [
                (block, room) for block, room in restricted_domain
                if room == 'Online'
            ]

        # Regista a variável no problema com o domínio já restringido
        problem.addVariable(var, restricted_domain)

    # --- 5. HARD CONSTRAINT IMPLEMENTATION ---
    # A partir daqui, adicionamos as restrições “duras” (obrigatórias).

    # Constraint A: Class Conflicts (A class can only have one lesson per block)
    # Para cada turma, duas aulas não podem cair no mesmo bloco temporal.
    for class_id in CLASS_COURSES.keys():
        # Pega todas as variáveis dessa turma
        class_lessons = [var for var in LESSON_VARIABLES if var.startswith(f'{class_id}_')]

        # Para cada par de variáveis da mesma turma, proíbe que o bloco (a[0]) coincida
        for i in range(len(class_lessons)):
            for j in range(i + 1, len(class_lessons)):
                var_a = class_lessons[i]
                var_b = class_lessons[j]
                # a e b são tuplas (block, room); a[0] é o bloco temporal
                problem.addConstraint(lambda a, b: a[0] != b[0], (var_a, var_b))

    # Constraint B: Teacher Conflicts (A teacher can only teach one lesson per block)
    # O mesmo professor não pode dar duas aulas ao mesmo tempo (mesmo bloco).
    for teacher, courses in TEACHER_COURSES.items():
        # Seleciona as variáveis cuja disciplina pertence ao conjunto que o professor leciona
        teacher_lessons = [
            var for var in LESSON_VARIABLES
            if get_lesson_info(var)[1] in courses
        ]

        # Garante que nenhum par de aulas desse professor acontece no mesmo bloco
        for i in range(len(teacher_lessons)):
            for j in range(i + 1, len(teacher_lessons)):
                var_a = teacher_lessons[i]
                var_b = teacher_lessons[j]
                problem.addConstraint(lambda a, b: a[0] != b[0], (var_a, var_b))

    # Constraint C: Room Conflicts (A physical room can only host one lesson per block)
    # Salas físicas não podem receber duas aulas no mesmo bloco (Online pode repetir).

    def room_overlap_constraint(a, b):
        block_a, room_a = a       # a é (bloco, sala)
        block_b, room_b = b       # b é (bloco, sala)

        # Se for o mesmo bloco e a mesma sala física -> conflito (proibido)
        if block_a == block_b and room_a == room_b and room_a in PHYSICAL_ROOMS:
            return False

        return True  # Caso contrário, está ok

    # Gera todos os pares possíveis de variáveis para aplicar a restrição de sala
    all_lesson_pairs = []
    for i in range(len(LESSON_VARIABLES)):
        for j in range(i + 1, len(LESSON_VARIABLES)):
            all_lesson_pairs.append((LESSON_VARIABLES[i], LESSON_VARIABLES[j]))

    # Adiciona a constraint de overlap de sala para cada par de variáveis
    for var_a, var_b in all_lesson_pairs:
        problem.addConstraint(room_overlap_constraint, (var_a, var_b))

    # Soft Constraint Implementation (Distinct Days) - Implementado como Hard Constraint
    # Aqui, pedimos que as duas aulas da MESMA disciplina para a MESMA turma
    # fiquem em dias diferentes (para espaçar no calendário).
    def distinct_days_constraint(a, b):
        day_a = get_day(a[0])  # a[0] é o bloco; get_day converte bloco -> dia
        day_b = get_day(b[0])
        return day_a != day_b  # Só aceita se forem dias distintos

    # Aplica a restrição para cada turma e cada disciplina dessa turma
    for class_id in CLASS_COURSES.keys():
        for course in CLASS_COURSES[class_id]:
            var_a = f'{class_id}_{course}_L1'
            var_b = f'{class_id}_{course}_L2'
            problem.addConstraint(distinct_days_constraint, (var_a, var_b))

    # --- 6. SOLVE AND DISPLAY RESULTS ---
    # Tenta resolver e, se conseguir, mostra o horário.

    print(f"\nTotal Lesson Variables to schedule: {len(LESSON_VARIABLES)}")
    print("Tentando encontrar um horário viável...")

    solutions = problem.getSolution()  # Procura uma solução (qualquer uma viável)

    if solutions:
        print("\n Solução de Horário Viável Encontrada. Gerando visualizações...\n")

        # Para cada turma, desenha o horário a partir da solução encontrada
        for class_id in sorted(CLASS_COURSES.keys()):
            display_timetable(solutions, class_id, CLASS_COURSES, COURSE_TEACHERS)

    else:
        # Se não encontrar solução, avisa. Normalmente faltam recursos ou sobram restrições.
        print("\n Nenhuma solução encontrada que satisfaça todas as restrições com os recursos dados.")


### Resolução com Monitorização de Desempenho

Nesta parte integrámos a função de resolução do CSP com o monitor de desempenho, de modo a avaliar o comportamento do sistema durante a execução.  
Enquanto o agente procura uma solução para o horário, são recolhidas leituras periódicas do uso de CPU e memória, o que nos permite perceber o impacto computacional do processo.  
Decidimos incluir esta monitorização para obter dados quantitativos sobre o desempenho do solver e comparar a eficiência entre diferentes execuções ou configurações.  
Além disso, é medido o tempo total de execução, informação útil para a análise final.


In [ ]:
def solve_and_display_monitored():
    # Iniciar monitoramento de desempenho (CPU e memória)
    # Esta função devolve um dicionário com medições feitas periodicamente.
    # O intervalo de 0.5s significa que a cada meio segundo recolhe uma leitura.
    perf_data = monitor_performance(interval=0.5)

    # Marca o tempo de início para calcular quanto tempo o processo de resolução demora.
    start = time.time()

    try:
        # Chama a função principal de resolução e visualização do CSP.
        # Esta parte é onde o problema de horários é realmente resolvido.
        solve_and_display()
    finally:
        # Mesmo que algo corra mal dentro de solve_and_display(),
        # este bloco "finally" garante que medimos e mostramos o tempo decorrido.
        end = time.time()
        print(f"\nTempo total de execução: {end - start:.2f}s")

    # Devolve os dados de desempenho recolhidos (CPU e memória ao longo do tempo)
    # Assim podemos depois analisar ou visualizar o comportamento do sistema.
    return perf_data


### Execução Principal do Sistema

Este bloco é responsável por iniciar todo o processo de criação e resolução do horário.  
O programa pede ao utilizador o ficheiro de dados, carrega as informações e define as variáveis globais do problema, como dias, blocos e salas.  
De seguida, é executada a função de resolução com monitorização de desempenho, que recolhe dados de CPU e memória, e no final são gerados gráficos que mostram a utilização de recursos durante a execução e a solução de horário para cada turma.

In [ ]:
if __name__ == "__main__":
    # --- EXECUÇÃO: CARREGAMENTO DE DADOS ---
    # Este bloco é executado apenas quando o script é rodado diretamente (não quando é importado).
    # Aqui começa todo o processo: pedir o ficheiro, carregar dados, resolver o CSP e mostrar resultados.

    try:
        print("\nPor favor, carregue o arquivo de dados (ex: ClassTT_01_tiny.txt):")
        # Em ambiente tipo Google Colab, isto abre o seletor de ficheiros para upload.
        filepath_dict = files.upload()

        # Se o utilizador não carregar nada, lançamos um erro explícito.
        if not filepath_dict:
            raise ValueError("Nenhum arquivo carregado.")

        # Extrai o nome do primeiro (e normalmente único) ficheiro carregado.
        data_filename = list(filepath_dict.keys())[0]

        # Chama a função que lê e processa o ficheiro, transformando em dicionários organizados.
        DATA = load_timetable_data(data_filename)

    except Exception as e:
        # Se algo correr mal (ficheiro inexistente, formato errado, etc.), entra aqui.
        print(f"\n--- ERRO CRÍTICO DE CARREGAMENTO DE DADOS ---")
        print(f"O script não pôde carregar ou processar o arquivo. Detalhes: {e}")

        # Garante que o programa não quebra totalmente, criando dicionários vazios de fallback.
        DATA = {
            'CLASS_COURSES': {},
            'TEACHER_COURSES': {},
            'COURSE_ROOM_RESTRICTIONS': {},
            'TEACHER_UNAVAILABLE_SLOTS': {},
            'ONLINE_LESSONS': {},
            'COURSE_TEACHERS': {}
        }

    # --- 1. CONSTANTES E VARIÁVEIS GLOBAIS ---
    # Estas constantes definem o "mundo" do problema — dias, blocos, salas, etc.

    DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']    # Dias da semana
    BLOCKS = list(range(1, 21))                   # 1–20 blocos no total (5 dias × 4 blocos/dia)
    ROOMS = ['R1', 'R2', 'R3', 'R4', 'Lab01', 'Online']  # Lista de todas as salas possíveis
    PHYSICAL_ROOMS = [r for r in ROOMS if r != 'Online']  # Salas físicas (tira a 'Online')

    # Extrai os conjuntos de dados carregados para variáveis globais,
    # para facilitar o uso nas funções seguintes.
    CLASS_COURSES = DATA['CLASS_COURSES']
    COURSE_TEACHERS = DATA['COURSE_TEACHERS']
    TEACHER_UNAVAILABLE_SLOTS = DATA['TEACHER_UNAVAILABLE_SLOTS']
    COURSE_ROOM_RESTRICTIONS = DATA['COURSE_ROOM_RESTRICTIONS']
    ONLINE_LESSONS = DATA['ONLINE_LESSONS']
    TEACHER_COURSES = DATA['TEACHER_COURSES']

    # Feedback visual ao utilizador
    print(f"\nDados carregados e {len(CLASS_COURSES)} turmas prontas para agendamento.")

    # Executa o solver e monitoriza o desempenho ao mesmo tempo.
    perf_data = solve_and_display_monitored()

    # Extrai os dados de desempenho para listas separadas (tempo, CPU, memória)
    times = list(perf_data.keys())
    cpu = [v["cpu"] for v in perf_data.values()]
    mem = [v["mem"] for v in perf_data.values()]

    # Cria o gráfico de uso de CPU e memória ao longo do tempo
    plt.figure(figsize=(10, 6))
    plt.plot(times, cpu, label="CPU (%)", linewidth=2)
    plt.plot(times, mem, label="Memória (%)", linewidth=2)

    plt.title("Uso de CPU e Memória durante execução")
    plt.xlabel("Tempo (s)")
    plt.ylabel("Uso (%)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()



Por favor, carregue o arquivo de dados (ex: ClassTT_01_tiny.txt):
